In [4]:
import pandas as pd

# Load the CSV file into a pandas DataFrame
file_path = 'trades_output.csv'  # Replace with your actual file path
trades_df = pd.read_csv(file_path)

# Convert the Entry Time and Exit Time to datetime objects
trades_df['Entry Time'] = pd.to_datetime(trades_df['Entry Time'])
trades_df['Exit Time'] = pd.to_datetime(trades_df['Exit Time'])

# Sort the DataFrame by Entry Time
trades_df = trades_df.sort_values(by='Entry Time').reset_index(drop=True)

# Check for overlapping trades
overlapping_trades = []

for i in range(1, len(trades_df)):
    if trades_df.loc[i, 'Entry Time'] < trades_df.loc[i-1, 'Exit Time']:
        overlapping_trades.append((i-1, i))

overlapping_trades_df = pd.DataFrame(overlapping_trades, columns=['Previous Trade Index', 'Current Trade Index'])

# Display overlapping trades
print(overlapping_trades_df)


Empty DataFrame
Columns: [Previous Trade Index, Current Trade Index]
Index: []


In [5]:
import pandas as pd

# Load the CSV files into pandas DataFrames
trades_file_path = 'trades_output.csv' # Replace with your file path
bars_file_path = 'bars.csv'  # Replace with your file path
trades_df = pd.read_csv(trades_file_path)
bars_df = pd.read_csv(bars_file_path)

# Convert the Entry Time and Exit Time to datetime objects in trades_df
trades_df['Entry Time'] = pd.to_datetime(trades_df['Entry Time'])
trades_df['Exit Time'] = pd.to_datetime(trades_df['Exit Time'])

# Convert the timestamp to datetime objects in bars_df
bars_df['timestamp'] = pd.to_datetime(bars_df['timestamp'])

# Initialize lists to hold any mismatches found
mismatches = []

# Function to check prices using previous and following index
def check_prices(entry_time, exit_time, entry_price, exit_price):
    entry_match = bars_df.loc[(bars_df['timestamp'] == entry_time) & (bars_df['close'] == entry_price)]
    exit_match = bars_df.loc[(bars_df['timestamp'] == exit_time) & (bars_df['close'] == exit_price)]
    
    if entry_match.empty:
        # Check previous and following row by index for entry time
        entry_index = bars_df.index[bars_df['timestamp'] == entry_time].tolist()
        if entry_index:
            entry_index = entry_index[0]
            if entry_index > 0 and bars_df.iloc[entry_index - 1]['close'] == entry_price:
                entry_match = bars_df.iloc[[entry_index - 1]]
            elif entry_index < len(bars_df) - 1 and bars_df.iloc[entry_index + 1]['close'] == entry_price:
                entry_match = bars_df.iloc[[entry_index + 1]]

    if exit_match.empty:
        # Check previous and following row by index for exit time
        exit_index = bars_df.index[bars_df['timestamp'] == exit_time].tolist()
        if exit_index:
            exit_index = exit_index[0]
            if exit_index > 0 and bars_df.iloc[exit_index - 1]['close'] == exit_price:
                exit_match = bars_df.iloc[[exit_index - 1]]
            elif exit_index < len(bars_df) - 1 and bars_df.iloc[exit_index + 1]['close'] == exit_price:
                exit_match = bars_df.iloc[[exit_index + 1]]

    return entry_match, exit_match

# Check each trade for matching prices
for index, row in trades_df.iterrows():
    entry_time = row['Entry Time']
    exit_time = row['Exit Time']
    entry_price = row['Entry Price']
    exit_price = row['Exit Price']
    
    entry_match, exit_match = check_prices(entry_time, exit_time, entry_price, exit_price)
    
    if entry_match.empty or exit_match.empty:
        mismatches.append({
            'ID': row['ID'],
            'Entry Time': entry_time,
            'Entry Price': entry_price,
            'Exit Time': exit_time,
            'Exit Price': exit_price,
            'Entry Match Found': not entry_match.empty,
            'Exit Match Found': not exit_match.empty
        })

# Convert mismatches to a DataFrame
mismatches_df = pd.DataFrame(mismatches)

# Display the mismatched trades
print(mismatches_df)


    ID          Entry Time  Entry Price           Exit Time  Exit Price  \
0    0 2024-06-03 12:05:00      79.3100 2024-06-03 12:06:00     79.2220   
1    1 2024-06-03 12:17:00      77.9100 2024-06-03 12:26:00     78.0000   
2    2 2024-06-03 12:33:00      78.0820 2024-06-03 13:00:00     78.1471   
3    3 2024-06-03 13:02:00      78.1600 2024-06-03 13:42:00     78.9006   
4    4 2024-06-03 13:48:00      78.9300 2024-06-03 13:50:00     78.8920   
5    5 2024-06-03 13:52:00      78.9000 2024-06-03 13:54:00     78.8440   
6    6 2024-06-03 13:56:00      78.9601 2024-06-03 14:08:00     79.2220   
7    7 2024-06-03 14:13:00      79.2100 2024-06-03 14:21:00     79.1608   
8    8 2024-06-03 14:23:00      79.2300 2024-06-03 14:30:00     79.1650   
9    9 2024-06-03 14:32:00      79.1900 2024-06-03 14:36:00     79.1630   
10  12 2024-06-03 14:50:00      79.1550 2024-06-03 14:53:00     79.0990   
11  13 2024-06-03 14:57:00      79.1600 2024-06-03 15:05:00     79.2140   
12  14 2024-06-03 15:06:0